QuickView

此文本可以部署到服务器，然后通过ip:port进行连接 实现数据的快速查看
此代码主要是通过读取文件夹里的图片信息进行整合，输出一个video 方便对数据集进行选择，以下我们会演示一个读取zod的文件夹内图片信息，然后输出视频

In [1]:
import os, glob
from tqdm import tqdm
from pathlib import Path
import numpy as np
import cv2

BASE_FOLDER = '/home/kin/data/zod'
read_folder_name = 'drives' # Let's say ther are 000000-000028 folders
img_folder_name = 'camera_front_blur' # all seqences have this folder
save_video_folder_name = 'preview_video'
view_single_img_t = 0.01 # [500ms, 2fps, since we need time to view data to check] if it's too slow, change to 10fps

# 1. Read the folder
for one_seq in sorted(os.listdir(os.path.join(BASE_FOLDER, read_folder_name))):
    img_array = []
    full_path = os.path.join(BASE_FOLDER, read_folder_name, one_seq, img_folder_name)
    if os.path.exists(os.path.join(BASE_FOLDER, save_video_folder_name, one_seq)+'.mp4') == True:
        print('Skip: ', os.path.join(BASE_FOLDER, save_video_folder_name, one_seq)+'.mp4', ' since it exists')
        continue
    total_img_num = len(os.listdir(full_path))
    skip_flag = True if total_img_num>1000 else False
    for i, one_img in tqdm(enumerate(sorted(os.listdir(full_path))), desc='seq: '+one_seq, total=total_img_num):
        if skip_flag == True and i%5 != 0:
            continue
        img_path = os.path.join(full_path, one_img)
        img = cv2.imread(img_path)
        # if you want to concat multiple images
        # im_v2 = cv2.vconcat([cv2.resize(fv,(1024,1024)), cv2.resize(right,(1024,768))])
        # full_image = cv2.hconcat([im_v1, im_v2])
        full_image = img
        height, width, layers = full_image.shape
        cv2.putText(full_image, one_seq+one_img, (10,100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 0), 2, cv2.LINE_AA)
        img_array.append(full_image)
    size = (width,height)
    
    # mkdir if not exist
    Path(os.path.join(BASE_FOLDER, save_video_folder_name)).mkdir(parents=True, exist_ok=True)
    out = cv2.VideoWriter(os.path.join(BASE_FOLDER, save_video_folder_name, one_seq)+'.mp4', cv2.VideoWriter_fourcc(*'VP90'), 1/view_single_img_t, size)
    for i in tqdm(range(len(img_array))):
        out.write(img_array[i])
    out.release()
    print('Done and check here: ', os.path.join(BASE_FOLDER, save_video_folder_name, one_seq)+'.mp4')

seq: 000000: 3212it [00:21, 147.75it/s]
OpenCV: FFMPEG: tag 0x30395056/'VP90' is not supported with codec id 167 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x39307076/'vp09'
  5%|▌         | 33/643 [00:15<04:53,  2.08it/s]


KeyboardInterrupt: 

In [ ]:
from ipywidgets import Video, Image
new_ = Video.from_file(os.path.join(BASE_FOLDER, save_video_folder_name, one_seq)+'.mp4',play=True,width=1400, height=1400)
new_